In [15]:
import numpy as np
import cv2
from glob import glob
import os
from natsort import natsorted
import matplotlib.pyplot as plt
import fly_analysis as fa
from scipy.signal import savgol_filter
from tqdm import tqdm
import pandas as pd

In [22]:
dt = 1/500
videos_folder = "/home/buchsbaum/Videos/20230801_134131/"
video_files = natsorted(glob(os.path.join(videos_folder, "*.mp4")))

In [10]:
def process_video(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Get total number of frames
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Initialize numpy array with NaN values
    results = np.full((total_frames, 4), np.nan)
    # Fill the first column with frame numbers
    results[:, 0] = np.arange(total_frames)
    
    frame_number = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Apply threshold
        _, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if contours:
            # Get the largest contour
            largest_contour = max(contours, key=cv2.contourArea)
            
            # Try to fit an ellipse
            if len(largest_contour) >= 5:  # Need at least 5 points to fit an ellipse
                ellipse = cv2.fitEllipse(largest_contour)
                
                # Extract information
                (x, y), (MA, ma), angle = ellipse
                
                # Update the corresponding row in results
                results[frame_number, 1:] = [x, y, angle]
        
        frame_number += 1
    
    # Release the video capture object
    cap.release()
    
    return results

In [12]:
for video in tqdm(video_files):
    results = process_video(video)
    np.savetxt(video + ".csv", results, delimiter=",")
    #print(f"Processed {video}")

100%|██████████| 724/724 [17:45<00:00,  1.47s/it]


In [14]:
csv_files = natsorted(glob(os.path.join(videos_folder, "*.csv")))
print(len(csv_files))

724


In [21]:
def sg_smooth(data, window_length=21, polyorder=3):
    return savgol_filter(data, window_length, polyorder)

In [20]:
for csv_file in tqdm(csv_files):
    df = pd.read_csv(csv_file, header=None, names=["frame", "x", "y", "angle"])
    x, y, angle = sg_smooth(df["x"].values), sg_smooth(df["y"].values), sg_smooth(df["angle"].values)
    xvel = np.gradient(x, dt)
    yvel = np.gradient(y, dt)
    linear_velocity = np.sqrt(xvel**2 + yvel**2)
    theta = np.arctan2(yvel, xvel)
    theta_u = fa.helpers.unwrap_ignore_nan(theta)
    angular_velocity = np.gradient(theta_u, dt)
    angular_velocity_degrees = np.rad2deg(angular_velocity)

    fig, axs = plt.subplots(ncols=2, nrows=2, figsize=(15, 5))
    axs = axs.flatten()

    axs[0].plot(x, y)
    axs[0].set_title("Trajectory")
    axs[0].set_xlabel("X (pixels)")
    axs[0].set_ylabel("Y (pixels)")

    axs[1].plot(df["frame"], linear_velocity)
    axs[1].set_title("Linear velocity")
    axs[1].set_xlabel("Frame")
    axs[1].set_ylabel("Velocity (pixels/s)")

    axs[2].plot(df["frame"], angular_velocity_degrees)
    axs[2].set_title("Angular velocity")
    axs[2].set_xlabel("Frame")
    axs[2].set_ylabel("Velocity (degrees/s)")

    axs[3].plot(df["frame"], angle)
    axs[3].set_title("Angle")
    axs[3].set_xlabel("Frame")
    axs[3].set_ylabel("Angle (degrees)")

    plt.tight_layout()
    plt.savefig(csv_file + ".png")
    plt.close('all')

  5%|▍         | 34/724 [00:12<04:38,  2.47it/s]/home/buchsbaum/.local/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:1303: RuntimeWarning: invalid value encountered in subtract
  out[tuple(slice1)] = (f[tuple(slice4)] - f[tuple(slice2)]) / (2. * ax_dx)
 38%|███▊      | 274/724 [01:32<02:21,  3.18it/s]/home/buchsbaum/.local/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:1303: RuntimeWarning: invalid value encountered in subtract
  out[tuple(slice1)] = (f[tuple(slice4)] - f[tuple(slice2)]) / (2. * ax_dx)
 52%|█████▏    | 376/724 [02:05<01:58,  2.95it/s]